In [1]:
import pandas as pd

In [ ]:
#  example
#  wrong letting pandas guess
from numpy import dtype
from pandas._typing import DtypeArg


df = pd.read_csv("data.csv")

# you provide explicit dtypes
dtype_spec:DtypeArg = {
    'user_id': 'Int64',
    'user_name': 'string',
    'age': 'Int64',
    'salary': 'Float64',
    'is_active': 'boolean'
}

df = pd.read_csv('data.csv',dtype=dtype_spec)

In [6]:
# Example: Without dtype specification
# CSV contains: user_id,score
#               001,95
#               002,88

df_inferred = pd.read_csv('scores.csv')
print(df_inferred.dtypes)
# user_id: int64  (leading zeros lost!)
# score: int64
# # With explicit dtypes
df_explicit = pd.read_csv('scores.csv', dtype={'user_id': 'string', 'score': 'int64'})
print(df_explicit['user_id'])
df_explicit
# # 0    001
# # 1    002
# # Preserves leading zeros


user_id    int64
score      int64
dtype: object
0    001
1    002
Name: user_id, dtype: string


,user_id,score
0,001,95
1,002,88


In [9]:
# from xmlrpc.client import boolean


pd.Series([True, False, None], dtype="boolean")
#  this is massive issue
# object = “I cannot represent this data efficiently, so I’ll store generic Python objects.”


0     True
1    False
2     <NA>
dtype: boolean

In [10]:
# <!--  working with datetime  -->
pd.Series(
    ["2024-01-01", "2024-01-02", None],
    dtype="datetime64[ns]"
)

0   2024-01-01
1   2024-01-02
2          NaT
dtype: datetime64[ns]

In [ ]:
df = pd.read_csv("dates_test.csv",parse_dates=['date'])
# df['date_parsed'] = pd.to_datetime(df['date'],errors='coerce', infer_datetime_format=True)
# df
df['date_parsed'] = pd.to_datetime(
    df['date'],
    errors='coerce',
    dayfirst=False,
    # infer_datetime_format=True
)

df


,id,date,value,date_parsed
0,1,2025-12-22,100,2025-12-22
1,2,2025/12/21,200,NaT
2,3,22-12-2025,150,NaT
3,4,Dec 21 2025,175,NaT
4,5,2025-13-01,120,NaT
5,6,2025-12-32,130,NaT
6,7,NaN,140,NaT
7,8,not_a_date,110,NaT
8,9,2025-12-20,160,2025-12-20
9,10,20251219,180,NaT
